In [1]:
from langchain_google_vertexai import ChatVertexAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)

from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers.json import SimpleJsonOutputParser

from typing import List, Tuple

import os
import json
import random

# Constants

In [9]:
START_INDEX = 0

# Data

In [2]:
PATH = "..\\..\\src\\data\\komus\\dataset.json"

with open(PATH, "r", encoding="UTF-8") as file:
    data = json.load(file)

len(data)

83755

In [ ]:
CATEGORIES_PATH = ".\\categories.txt"

with open(CATEGORIES_PATH, "r", encoding="UTF-8") as file:
    categories_array = file.read().splitlines()

categories_txt = "\n".join([f"{i}. {category}" for i, category in enumerate(categories_array, start=1)])

# Model

In [11]:
from langchain_openai import ChatOpenAI

T_PRO_CREDS = "..\\..\\secrets\\t-pro.json"

with open(T_PRO_CREDS) as file:
    model_params = json.load(file)

llm = ChatOpenAI(**model_params, temperature=0)

# Utils

In [34]:
def transform_data(data, **kwargs):
    """
    Преобразует массив словарей, оставляя только нужные ключи.

    :param data: Исходный массив словарей.
    :return: Новый массив словарей с преобразованными данными.
    """
    transformed_data = []
    for item in data:
        new_item = {
            "categories": categories_txt,
            "format_instructions": kwargs["format_instructions"],
            "problem_title": item.get("title", None),
            "problem_categories": item["category"]
        }
        transformed_data.append(new_item)
    return transformed_data

In [5]:
# Функция для создания батчей
def create_batches(array, batch_size, start_index=0):
    """
    Разделяет массив на батчи заданного размера, начиная с указанного индекса.

    :param array: Исходный массив (список).
    :param batch_size: Размер одного батча.
    :param start_index: Индекс, с которого начинать создание батчей.
    :return: Список батчей (списков).
    """
    if batch_size <= 0:
        raise ValueError("Размер батча должен быть больше 0.")
    # Обрезаем массив, начиная с start_index
    array = array[start_index:]
    batches = [array[i:i + batch_size] for i in range(0, len(array), batch_size)]
    return batches

In [6]:
def create_model_prompts(system_prompt: str, user_prompt: str) -> ChatPromptTemplate:
    """
    Создает шаблон промпта для модели.

    :param system_prompt: Системный промпт.
    :param user_prompt: Пользовательский промпт.
    :return: Шаблон промпта.
    """
    system_prompt_template = SystemMessagePromptTemplate.from_template(system_prompt)
    user_prompt_template = HumanMessagePromptTemplate.from_template(user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages([system_prompt_template, user_prompt_template])
    return chat_prompt

# Prompts

In [8]:
class ClassifyResponse(BaseModel):
    category: str = Field(..., description="""Ответ в формате: КАТЕГОРИЯ""")

parser = PydanticOutputParser(pydantic_object=ClassifyResponse)

SYSTEM_PROMPT = """
Ты — эксперт в классификации товаров. Твоя задача — определить категорию товара на основе его названия и вложенных категорий. Используй следующие категории:

{categories}

Правила классификации:  
- Внимательно анализируй название товара.  
- Если название явно указывает на категорию, выбери соответствующую.  
- Если категория не очевидна, выбери наиболее подходящую из предложенных.
- Если в названии товарной позиции описывается действие, то это должна быть категория "Услуги"
- ПО считается услугой


Правила для формирования ответа:
1. **Финальный ответ должен быть представлен в виде JSON-объекта, заключённого в один блок кода.**  
   - Финальный ответ должен быть корректным JSON-объектом, который заключается в один блок кода (используя три обратных апострофа ```).
   - Внутри JSON-объекта не должно быть дополнительных объяснений или текстовых комментариев.
2. **Фраза "Окончательный ответ" должна предварять блок кода с JSON-объектом.**  
   - Перед блоком кода с JSON-объектом должна быть написана фраза **"Окончательный ответ:"** (без кавычек в самой фразе).  
   - Между фразой и блоком кода не должно быть пустых строк.

Формат ответа должен соответствовать этому: {format_instructions}
""".strip()

USER_PROMPT = """
Товарная позиция: {problem_title}
Вложенные категории: {problem_categories}
""".strip()

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

# Transform data

In [41]:
transformed_data = transform_data(data, format_instructions=parser.get_format_instructions())

batches = create_batches(transformed_data, batch_size=100, start_index=START_INDEX)

In [12]:
from os import mkdir

mkdir("dataset")

In [13]:
def get_category(item, parser, categories_array):
    if "Окончательный ответ:" not in item.content:
        return ""
    content_part = item.content.split("Окончательный ответ:")[1]
    parsed = parser.invoke(content_part)
    return parsed.category if parsed.category in categories_array else ""

In [42]:
chain = prompt | llm

for i, batch in enumerate(batches, start=START_INDEX):
    results = chain.batch(batch) 

    parsed_results = [get_category(item, parser, categories_array) for item in results]

    dump_data = [{
        "title": product["problem_title"],
        "categories": product["problem_categories"],
        "answer": res
    } for product, res in zip(batch, parsed_results)]

    with open(f".\\dataset\\batch_{i}.json", "w", encoding="UTF-8") as file:
        print(f"Save: {file.name}")
        json.dump(dump_data, file, ensure_ascii=False, indent=4)

Save: .\dataset\batch_0.json
Save: .\dataset\batch_1.json
Save: .\dataset\batch_2.json
Save: .\dataset\batch_3.json
Save: .\dataset\batch_4.json
Save: .\dataset\batch_5.json
Save: .\dataset\batch_6.json
Save: .\dataset\batch_7.json
Save: .\dataset\batch_8.json
Save: .\dataset\batch_9.json
Save: .\dataset\batch_10.json
Save: .\dataset\batch_11.json
Save: .\dataset\batch_12.json
Save: .\dataset\batch_13.json
Save: .\dataset\batch_14.json
Save: .\dataset\batch_15.json
Save: .\dataset\batch_16.json
Save: .\dataset\batch_17.json
Save: .\dataset\batch_18.json
Save: .\dataset\batch_19.json
Save: .\dataset\batch_20.json
Save: .\dataset\batch_21.json
Save: .\dataset\batch_22.json
Save: .\dataset\batch_23.json
Save: .\dataset\batch_24.json
Save: .\dataset\batch_25.json
Save: .\dataset\batch_26.json
Save: .\dataset\batch_27.json
Save: .\dataset\batch_28.json
Save: .\dataset\batch_29.json
Save: .\dataset\batch_30.json
Save: .\dataset\batch_31.json
Save: .\dataset\batch_32.json
Save: .\dataset\batc

: 